# 达闼杯——机器人大模型与具身智能挑战赛

导入必要的包

In [9]:
import time
import random

In [11]:
from utils.SceneManager import SceneManager
from utils.PedestrianController import PedestrianController

## 任务1 环境主动探索和记忆

In [12]:
#生成环境的语义地图

## 任务2 视觉语言导航

识别顾客靠近，通过对话结合大模型为行人匹配最合适的座位，并正确指引顾客到指定位置

In [1]:
import math
import time
import random

from utils.RobotTaskController import RobotTaskController
from utils.SceneManager import SceneManager
from utils.PedestrianController import PedestrianController
from utils.NavigationController import NavigationController

from VLN_find_chair.model.InquireChairNeeds import *
from VLN_find_chair.model.match_best_chair import *



'''场景一：咖啡厅服务员位于吧台处等待，识别顾客靠近，为行人匹配座位'''


def distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def check_proximity(pos, obstacles, min_distance=100):  # 假设最小安全距离是100
    for obstacle_pos in obstacles:
        if distance(pos, obstacle_pos) < min_distance:
            return False  # 如果距离太近，返回False
    return True

# 定义一个函数用来检测顾客靠近的操作
def detect_customer_proximity(walkers):

    for walker in walkers:
        robot_x = scene.location.X
        robot_y = scene.location.Y

        detection_range = 200

        walker_x, walker_y = walker.pose.X, walker.pose.Y

        distance = ((walker_x - robot_x) ** 2 + (walker_y - robot_y) ** 2) ** 0.5
        if distance <= detection_range:
            detected_customer = walker
            return detected_customer

'''
1. 场景生成
'''
 # Create an instance of the SceneManager class
scene_manager = SceneManager()
map_id = 11    # 地图编号
scene_num = 1  # 场景数量
print('------------ 初始化加载场景 ------------')
scene_manager.Init()
scene_manager.AcquireAvailableMaps()
scene_manager.SetWorld(map_id, scene_num)
time.sleep(5.0)
print('------------ 场景操作 ------------')
scene_manager.Reset(0)
scene=scene_manager.Observe(0)

'''
2. 添加随机行人
'''
pedestrian_min = 3
pedestrian_max = 15
pedestrian_count = random.randint(pedestrian_min,pedestrian_max)
print(pedestrian_count)

#场景坐标的范围
scene_bounds_x = [-700, 700]
scene_bounds_y = [-1400, 1400]

#生成行人坐标
pedestrian_data = []
# #障碍物的位置
# obstacle_positions = []
# for obj in scene.objects:
#     obstacle_positions.append((obj.location.X, obj.location.Y))
# # 与场景中的物体坐标进行判断，生成可以生成行人的位置坐标
# for walker_id in range(pedestrian_count):
#     valid_position = False
#     while not valid_position:
#         start_x = random.uniform(scene_bounds_x[0], scene_bounds_x[1])
#         start_y = random.uniform(scene_bounds_y[0], scene_bounds_y[1])
#         start_yaw = random.uniform(0, 360)
#         if check_proximity((start_x, start_y), obstacle_positions):
#             valid_position = True
#             pedestrian_data.append([walker_id, start_x, start_y, start_yaw])
# print(pedestrian_data)
# 先暂时改成固定点
pedestrian_data = [[0, 0, 880, 0], [1, 250, 1200,45], [2, -55, 750,90], [3, 70, -200,180]]

#添加行人
pedestrian_controller = PedestrianController(scene_manager)
result_scene = pedestrian_controller.add_multiple_pedestrians(pedestrian_data, scene_id=0)

walkers = pedestrian_controller.get_walkers()
print(walkers)

# 生成有walker_id和walker索引的列表
walker_list = [{'walker_id': index, 'walker': walker} for index, walker in enumerate(walkers)]
print(walker_list)

#让所有行人都随便走
for pedestrian_data in walker_list:
    pedestrian_id = pedestrian_data['walker_id']
    auto_control=True
    pedestrian_controller.control_one_pedestrian_autowalk(pedestrian_id, autowalker=auto_control, walker_speed=50, scene_id=0)

# # 创建并启动一个线程来检测顾客靠近机器人
# stop_event = threading.Event()  # Event to stop the thread
# result_queue = queue.Queue()  # Queue to store detected customer name
#
# detection_thread = threading.Thread(target=detect_customer_proximity, args=(scene_manager, result_queue, stop_event))
# detection_thread.start()
'''
3. 某个行人靠近吧台
'''
# 延时5s以后
time.sleep(5)
# 从现有行人中随机选择一个走向吧台
selected_walker = random.choice(walker_list)
walker_id = selected_walker['walker_id']  # 获取选定行人的ID

# 确定吧台的坐标
bar_x = 100
bar_y = 530#在[520-540]范围内
end_yaw = 0

# # 生成其他行人的随机终点位置并让它们走向这些位置
# for pedestrian_data in walker_list:
#     if pedestrian_data['walker_id'] != walker_id:
#         pedestrian_id = pedestrian_data['walker_id']
#         x = random.uniform(scene_bounds_x[0], scene_bounds_x[1])
#         y = random.uniform(scene_bounds_y[0], scene_bounds_y[1])
#         yaw = random.uniform(0, 360)
#         pedestrian_controller.control_one_pedestrian(pedestrian_id, x, y, yaw, walker_speed=200, scene_id=0)
#     else:
#         # 控制选定的行人走向吧台
#         pedestrian_controller.control_one_pedestrian(walker_id, bar_x, bar_y, end_yaw, walker_speed=200, scene_id=0)
pedestrian_controller.control_one_pedestrian_autowalk(walker_id, autowalker=False, walker_speed=200, scene_id=0)
# 控制选定的行人走向吧台
pedestrian_controller.control_one_pedestrian(walker_id, bar_x, bar_y, end_yaw, walker_speed=200, scene_id=0)


'''
4. 识别到顾客靠近，向顾客问好，并询问具体的人数，座位需求
'''
# TODO:调用视觉模型，检测到顾客靠近
#现在的实现方式的话，你可以遍历walker中的每一个，判断是否在机器人附近

# 识别到顾客靠近
while True:
    current_scene = scene_manager.Observe(0)
    detected_customer = detect_customer_proximity(current_scene.walkers)
    if detected_customer:
        print(f"Detected customer : {detected_customer}")
        break
    else:
        print("No customer detected near the robot")
    time.sleep(1)  # 添加一个时间间隔

# 向顾客问好
robot_task_controller = RobotTaskController(scene_manager)
robot_task_controller.display_text_bubble("您好，您需要什么帮助吗？")
time.sleep(2)
talk_walker_response = " I'm here alone.I'd like a seat by the window."
pedestrian_controller.talk_walkers(detected_customer.name, talk_walker_response)
#执行输出
inquirer = InquireChairNeeds()  # 初始化对话类
# response = inquirer.initiate_conversation()  # 调用对话函数
res = inquirer.get_completion_from_messages(talk_walker_response)#开始匹配
print(res)
chair_list = ChairList()
ordered_feature = chair_list.decode_feature("{\n  \"seat_preference\": \"near window\",\n  \"number_of_people\": 4\n }")
chair = chair_list.find_the_best(ordered_feature)
print(chair)

#带领
navi = NavigationController(scene_manager)
print(navi.navigate_to_limit(chair["position"][0],chair["position"][1],end_yaw,200,100))
# Sleep(1000)
#更新椅子数目
time.sleep(5.0)
#行人走向目标椅子
pedestrian_controller.control_one_pedestrian(walker_id,chair["position"][0],chair["position"][1],end_yaw, walker_speed=200, scene_id=0)






ModuleNotFoundError: No module named 'InquireChairNeeds'

In [7]:
#执行输出
inquirer = InquireChairNeeds()  # 初始化对话类
response = inquirer.initiate_conversation()  # 调用对话函数
talk_walker_response = " I'm here alone.I'd like a seat by the fireplace for a warm and cozy ambiance."
res = inquirer.get_completion(talk_walker_response)#开始匹配
print(res)


AttributeError: module 'openai' has no attribute 'get_completion_from_messages'

'{
  "seat_preference": "near window",
  "number_of_people": 4
}'
结果好像不是很好

In [ ]:
chairSelector = ChairList()
chairSelector.encode_feature(res)
best = chairSelector.find_the_best()

### 场景一：咖啡厅服务员位于吧台处等待，识别顾客靠近，为行人匹配座位

1. 场景生成

In [13]:
 # Create an instance of the SceneManager class
scene_manager = SceneManager()

map_id = 11    # 地图编号

scene_num = 1  # 场景数量

print('------------ 初始化加载场景 ------------')
scene_manager.Init()
scene_manager.AcquireAvailableMaps()
scene_manager.SetWorld(map_id, scene_num)
time.sleep(5.0)

for i in range(scene_num):
    print('------------ 场景操作 ------------')
    scene_manager.Reset(i)
    scene=scene_manager.Observe(i)
    #print(scene)

------------ 初始化加载场景 ------------
maps {
}
maps {
}
maps {
}
maps {
}
maps {
  name: "Starbucks_Ginger1.0"
  width: 1400.0
  height: 2800.0
}
maps {
}
maps {
}
maps {
}
maps {
}
maps {
}
maps {
}
maps {
  name: "Starbucks_Ginger2.0"
  width: 1400.0
  height: 2800.0
}

------------------SetWorld----------------------
------------ 场景操作 ------------
------------------Reset----------------------
location {
  X: 247.0
  Y: 520.0
}
rotation {
  Yaw: 180.0
}
joints {
  name: "Knee_X"
  location {
    X: 246.98728942871094
    Y: 520.0
    Z: 43.66059494018555
  }
}
joints {
  name: "Back_Y"
  location {
    X: 246.97854614257812
    Y: 520.0
    Z: 73.71397399902344
  }
}
joints {
  name: "Back_X"
  location {
    X: 246.9785614013672
    Y: 520.0
    Z: 86.15656280517578
  }
}
joints {
  name: "Back_Z"
  location {
    X: 246.9785614013672
    Y: 520.0
    Z: 98.02046966552734
  }
}
joints {
  name: "Neck_Y"
  location {
    X: 247.37466430664062
    Y: 520.0
    Z: 128.04061889648438
  }
}


location:[X: 247.0
Y: 520.0
], rotation:Yaw: 180.0

 joints number:21, fingers number:10
 objects number: 344, walkers number: 0
timestep:0, timestamp:16998607182995559
collision:, info:


2. 添加随机行人

In [7]:
# # Generate a random number of pedestrians between 3 and 7
# pedestrian_count = random.randint(3, 7)
# print(pedestrian_count)

7


In [8]:
# pedestrian_data = []
# #TODO:确定场景边界
# scene_bounds_x=[-700,700]
# scene_bounds_y=[-1400,1400]
# for walker_id in range(pedestrian_count):
#     start_x = random.uniform(scene_bounds_x[0], scene_bounds_x[1])  # Assuming scene_bounds_x defines x boundaries
#     start_y = random.uniform(scene_bounds_y[0], scene_bounds_y[1])  # Assuming scene_bounds_y defines y boundaries
#     start_yaw = random.uniform(0, 360)  # Randomize yaw angle
#     pedestrian_data.append([walker_id, start_x, start_y, start_yaw])
# print(pedestrian_data)
# #TODO:场景中存在其他物体，不一定能够在随机生成的点的位置上生成人

[[0, -259.420198059731, -1216.6178305390472, 51.67637862904141], [1, -180.15647559396007, -250.03489222288317, 87.35462065866324], [2, 304.23662121767586, -891.6092498194478, 329.60193800181213], [3, -695.3583991207915, -771.7062866327284, 350.4718304807018], [4, 658.5813878624913, -938.6067864761408, 212.74588855465814], [5, 210.29150407593852, -291.81155229172873, 86.5104009495514], [6, -445.091329503715, -375.35292079394685, 96.0720471295291]]


In [14]:
# 暂时改成生成固定的行人
pedestrian_data = [[0, 0, 880, 0], [1, 250, 1200,45], [2, -55, 750,90], [3, 70, -200,180]]

In [15]:
pedestrian_controller = PedestrianController(scene_manager)
result_scene = pedestrian_controller.add_multiple_pedestrians(pedestrian_data, scene_id=0)
walkers=pedestrian_controller.get_walkers()
print(walkers)

[name: "Boy01_C_0"
pose {
  Y: 880.0
}
speed: 600.0
target {
  Y: 880.0
}
, name: "Boy02_C_0"
pose {
  X: 250.0
  Y: 1200.0
  Yaw: 0.3826834261417389
}
speed: 600.0
target {
  X: 250.0
  Y: 1200.0
  Yaw: 45.0
}
, name: "Boy03_C_0"
pose {
  X: -55.0
  Y: 750.0
  Yaw: 0.7071067690849304
}
speed: 600.0
target {
  X: -55.0
  Y: 750.0
  Yaw: 90.0
}
, name: "BoyEuro01_C_0"
pose {
  X: 70.0
  Y: -200.0
  Yaw: 1.0
}
speed: 600.0
target {
  X: 70.0
  Y: -200.0
  Yaw: 180.0
}
]


In [ ]:
#带领
navi = NavigationController(scene_manager)
navi.navigate_to(chair["position"][0],chair["position"][1],end_yaw)
#行人走向目标椅子
pedestrian_controller.control_one_pedestrian(walker_id, bar_x, bar_y, end_yaw, walker_speed=200, scene_id=0)



3. 某个行人靠近吧台

In [ ]:
#从walker中随机选一个行人靠近吧台
walker_id=walkers[0].id
#walker中的其他行人在场地中随意走动(随机生成几个终点位置，让其他行人走过去)


# 暂时改成0号行人靠近吧台
#TODO:确定吧台的坐标
pedestrian_controller.control_one_pedestrian(walker_id, end_x, end_y, end_yaw, walker_speed=200, scene_id=0)

4. 识别顾客靠近，向顾客问好，并询问具体的人数，座位需求

In [ ]:
#TODO:调用视觉模型，检测到顾客靠近
#现在的实现方式的话，你可以遍历walker中的每一个，判断是否在机器人附近

#向顾客问好，并询问具体的人数,座位需求
#TODO:封装好对询问椅子分配需求的类
inquirer = InquireChairNeeds()  # 初始化对话类
order_data = inquirer.initiate_conversation()  # 调用对话函数#这个应该就是答复的order列表

5. 行人给出答复，确定最佳座位匹配

In [ ]:
#调用ChairList类确定最佳座位分配
# TODO:优化和改进ChairList类
chair_list = ChairList(order_data)
best_chairs = chair_list.find_the_best()#确定输出的格式
chair_list.update_empty_chair_list(best_chairs)

6. 正确指引顾客到指定位置，任务结束

In [5]:
# TODO:根据数据集找到最佳椅子的坐标?（疑问就是如何找到椅子)
# 调用接口控制机器人走向正确的座位
# 调用文字冒泡输出：请您跟我来
# 调用接口控制行人走向正确的座位
# 调用行人的文字冒泡输出：谢谢
# 场景结束

### 场景二：咖啡服务员主动在场景内寻找有需求的顾客，走到顾客附近，为行人匹配座位

1. 场景生成

2. 添加行人


In [ ]:
# 添加行人以后，选择其中的一个行人冒泡:我该坐到哪里呀？发出请求

3. 服务员在咖啡厅内主动寻找附近最近没有落座的顾客，走到顾客附近

In [6]:
# TODO:调用机器人位姿预测的模型寻找附近最近的顾客
# 现在调用observe找到最近的文字冒泡的顾客

4. 向顾客问好，并询问具体的人数，座位需求

5. 行人给出答复，确定最佳座位匹配

6. 正确指引顾客到指定位置，任务结束

## 任务3 具身多轮对话